In [1]:
import os
import pandas as pd
import numpy as np
import requests
import io
import json
from datetime import datetime
import matplotlib 
import matplotlib.pyplot as plt


%run SLC_Functions.py

## Check out and read in the WL data

Home page link - https://uhslc.soest.hawaii.edu/
Data Dashboard  - https://uhslc.soest.hawaii.edu/reservoir/

####  Reservoir level data from the Sea Level center Reservoir Database gets pulled by Pull_WL_Data.ipynb


In [2]:
datapath = os.path.join("..", "Data/WL_Data2024-09-27/UHSLC")

# Create a metadata describe frame for the Reservoir data 
data_describe = create_metadata_from_SLC_Data(datapath)

data_describe.head()

C:\Users\bgorb\OneDrive\Documents\GitHub\SLC_Reservoir_Data\Scripts\SLC_Functions.py:30: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(os.path.join(datapath, file), parse_dates=['date'])
C:\Users\bgorb\OneDrive\Documents\GitHub\SLC_Reservoir_Data\Scripts\SLC_Functions.py:30: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(os.path.join(datapath, file), parse_dates=['date'])


,station_id,date_min,date_max,mean_WL,median_WL,min_WL,max_WL
0,EDD00214,2018-02-07 20:09:00+00:00,2024-09-27 21:04:00+00:00,1471.264692,1470.0,-129570.0,130758.0
1,EDD00CC6,2018-02-07 20:09:00+00:00,2024-09-27 21:04:00+00:00,939.506339,832.0,658.0,131071.0
2,EDD01162,2022-04-26 22:09:00+00:00,2024-09-27 21:04:00+00:00,15293.554955,-226.0,-324.0,131071.0
3,EDD024F8,2018-02-07 20:09:00+00:00,2024-09-27 21:04:00+00:00,38210.626583,38142.0,-108451.0,131071.0
4,EDD02A2A,2018-02-07 20:09:00+00:00,2024-09-27 21:04:00+00:00,96898.021513,97104.0,-74419.0,131071.0


## Convert to daily Water Level data 

In [3]:
datapath = os.path.join("..", "Data/WL_Data2024-09-27/UHSLC")

daily_dic = process_SLC_Data_To_daily(datapath)

C:\Users\bgorb\OneDrive\Documents\GitHub\SLC_Reservoir_Data\Scripts\SLC_Functions.py:75: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, parse_dates=['date'])


TypeError: agg function failed [how->mean,dtype->object]

# Read in the USGS Data


In [ ]:
USGS_directory = os.path.join("..", "Data/WL_Data2024-09-27/USGS")
daily_dic_USGS = process_USGS_Data_To_daily(USGS_directory)

## Get rid of the date part in the keys:
daily_dic_USGS = {key.split('--')[0] + '.csv': value for key, value in daily_dic_USGS.items()}

##### Merge SLC data and USGS Data 

In [ ]:
daily_dic.update(daily_dic_USGS)

## Read In the Precip data from HCDP 

In [ ]:
RainData = pd.read_csv(os.path.join("..", 'Data/HCDP_Rainfall_Daily_Data', "HCDP_RainData2018-2024.csv"))
RainData = RainData.set_index("index")   # make sure the index is date
RainData.index = pd.to_datetime(RainData.index, errors='coerce')


RainData_Meta = pd.read_csv(os.path.join("..", 'Data/HCDP_Rainfall_Daily_Data', "HCDP_Rain_METADATA_2018-2024.csv"))

# First make Arc read it as a string 
RainData_Meta["SKN"] = RainData_Meta["SKN"].astype(str)+"_R"  
RainData_Meta.to_csv(os.path.join("..", 'Data/HCDP_Rainfall_Daily_Data', "HCDP_Rain_METADATA_2018-2024_ArcrReady.csv"))

## Went into Arc to create the near table
- Used RainData_Meta_v2.csv, cleared out useless columns to make RainData_Meta_v2_Clean.csv. 
- Then manually merged UHSLC_Station_Metadata.csv, and USGS_ID_Meta.xlsx, so they only had the station names and lat lon and IDs. 
- In Arc brought in, turned to shapefiles, then near table tool on the Reservoirs, selecting only the 10 closest rain station points.  
- Arc only records the FIDs so have to do 2 joins on the original tables. 
- Result is Res_Rain_Near_Table_New_v2.csv, after 30 minutes of wasted time \

In [5]:
near_table = pd.read_csv(os.path.join("..", "Data/Metadata_Sheets", "Res_Rain_Near_Table_New_v2.csv"))

## Import metadata
Station_Meta = pd.read_csv(os.path.join("..", "Data/Metadata_Sheets", "ALL_HI_reservoirs_Metadata.csv"))


# Create plots of reservoir level and rainfall 

Use this for testing 
smalldic = {key: daily_dic[key] for key in ['EDD00214.csv', 'EDD00CC6.csv', 'EDD01FB0.csv', 'EDD024F8.csv']}  

In [7]:
#smalldic = {key: daily_dic[key] for key in ['EDD00214.csv', 'EDD00CC6.csv', 'EDD024F8.csv', 'USGS_213133158014201.csv']}   # For testing 

# Paramater that changes how many near rainfall stations to use 
Nearest_X_Stations = 10

for i in daily_dic.keys():  #daily_dic.keys(): 
    stat_id = i.split('.')[0]   # Pull off the .csv
    #### Modify for USGS 
    isla = Station_Meta['Res_ID'][Station_Meta['id'] == stat_id].values[0].split('-')[0]  # Pull out the island from the dlnr ID
    
    fig, ax = plt.subplots(figsize=(8,3))
    daily_dic[i].plot(y='data', ax=ax, marker=".", color='k', alpha=0.5, label="WL_(ft)")
    
    # Add the lever alert lines
    Level_alert_on = Station_Meta['level_alert_on'][Station_Meta['id'] == stat_id].values[0]
    plt.axhline(y=Level_alert_on, color='r', linestyle='-', alpha = 0.5)   
    Level_alert_off = Station_Meta['level_alert_off'][Station_Meta['id'] == stat_id].values[0]
    plt.axhline(y=Level_alert_off, color='g', linestyle='-', alpha = 0.5)
    
    
    nearest_SKN_table = near_table[near_table['ID'] == stat_id]     # Reference the nearest stations for rainfall
    nearest_X_SKNs =  nearest_SKN_table[nearest_SKN_table['NEAR_RANK'] <= Nearest_X_Stations]  # Cut to the number specified in Nearest_X_Stations
    nearest5skns = list(nearest_X_SKNs['SKN'])                      # Pull out the SKN vales from the dataframe 
    nearest5skns = list(map(lambda x: x.split("_")[0], nearest5skns))  # Quick list function map to pull of the "_R" from each SKN
    nearest5skns = list(map(lambda x: str(x), nearest5skns))  # Quick list function map to turn values back into str
    rain_datatoplot_5 = RainData[nearest5skns]     # make it a single dataframe 
    rain_datatoplot_5_mean = rain_datatoplot_5.mean(axis=1)    # calculate the average rainfall over all N stations 
    
    # Plot Rain Data
    ax2=ax.twinx()
    rain_datatoplot_5_mean.plot(ax=ax2, color='b', alpha=0.2)
    
    plt.title(stat_id +" "+ "{}".format(Station_Meta['Name'][Station_Meta['id'] == stat_id].values[0])+"--"+isla)
    ax2.set_ylabel('Rainfall (mm)', color='b')
    ax.set_ylabel('WL-reservoir (ft)', color='k')
    plt.savefig('res.png')
    plt.tight_layout()


NameError: name 'daily_WL_ommitted' is not defined